# Tutorial 03: Deposition MD (PVD Simulation)

**Goal**: Simulate the physical vapor deposition (PVD) process of Fe and Pt on MgO.

This tutorial demonstrates:
1.  **Deployment**: Using the trained potential in a production MD simulation.
2.  **Hybrid Potentials**: Combining the ML potential (PACE) with a physics baseline (ZBL) for robustness during high-energy collisions.
3.  **Complex Dynamics**: Simulating deposition using LAMMPS `fix deposit`.



In [ ]:
import os
import shutil
import subprocess
from pathlib import Path

from ase.io import read, write
from ase.visualize import view



## 1. Setup Environment

We need LAMMPS with the `MACHDYN` or `USER-MLIP` package installed to run `pair_style pace`.



In [ ]:
def has_command(cmd):
    return shutil.which(cmd) is not None

HAS_LAMMPS = has_command("lmp")
IS_CI_MODE = os.environ.get("IS_CI_MODE", "False").lower() == "true"

if not HAS_LAMMPS:
    print("LAMMPS not found. Forcing CI Mode.")
    IS_CI_MODE = True

WORKDIR = Path("workdirs/03_deposition")
if WORKDIR.exists():
    shutil.rmtree(WORKDIR)
WORKDIR.mkdir(parents=True, exist_ok=True)



## 2. Prepare Simulation

We need:
1.  **Initial Structure**: The MgO substrate (from Tutorial 02 or generated fresh).
2.  **Potential File**: The `.yace` file from training.
3.  **LAMMPS Input Script**: Configuring `fix deposit`.



In [ ]:
# 1. Generate Substrate (MgO 6x6x2)
from ase.build import bulk, surface
mgo = bulk("MgO", "rocksalt", a=4.21)
slab = surface(mgo, (1, 0, 0), 2, vacuum=20.0)
slab = slab.repeat((6, 6, 1))
slab.write(WORKDIR / "substrate.xyz")

print(f"Substrate created: {len(slab)} atoms")

# 2. Locate Potential
# In a real scenario, we would use the file from Tutorial 02.
# Here we define a path. If it doesn't exist (CI mode), we'll mock it.
POTENTIAL_PATH = Path("workdirs/02_interface/cycle_01/potential.yace").resolve()

if IS_CI_MODE and not POTENTIAL_PATH.exists():
    # Create a dummy potential file for CI
    POTENTIAL_PATH.parent.mkdir(parents=True, exist_ok=True)
    POTENTIAL_PATH.touch()



## 3. Generate LAMMPS Input

We write the input script dynamically. Note the use of `pair_style hybrid/overlay`.



In [ ]:
lammps_input = f"""
# PVD Simulation of Fe/Pt on MgO

units           metal
atom_style      atomic
boundary        p p f

# Read data (we will convert xyz to lammps data or read directly if using explicit creating)
# For simplicity in this script, we assume we can read a data file.
# But ASE writes data files easily.
"""

# Convert substrate to LAMMPS data
from ase.io.lammpsdata import write_lammps_data
write_lammps_data(WORKDIR / "data.substrate", slab, specorder=["Mg", "O", "Fe", "Pt"])

lammps_input += f"""
read_data       {WORKDIR.resolve() / 'data.substrate'}

# Define masses (approximate)
mass 1 24.305 # Mg
mass 2 15.999 # O
mass 3 55.845 # Fe
mass 4 195.084 # Pt

# --- Potential Setup ---
# Hybrid overlay: PACE (ML) + ZBL (Physics)
pair_style      hybrid/overlay pace zbl 0.5 2.0
"""

if IS_CI_MODE:
    # In CI mode, we can't run 'pace' if the library isn't loaded or potential is invalid.
    # We fallback to LJ for the sake of the script running without crashing LAMMPS (if lmp exists)
    # OR we just skip the execution.
    lammps_input += """
    # CI MODE: Fallback to LJ
    pair_style      lj/cut 2.5
    pair_coeff      * * 0.0 0.0 # No interaction for simplicity or dummy
    """
else:
    lammps_input += f"""
    pair_coeff      * * pace {POTENTIAL_PATH} Mg O Fe Pt
    pair_coeff      * * zbl 0.5 2.0 # ZBL for all pairs at short range
    """

lammps_input += """
# --- Deposition ---
region          slab block INF INF INF INF INF 10.0
group           substrate region slab

# Fix substrate bottom to prevent drifting
region          bottom block INF INF INF INF INF 2.0
group           fixed_bottom region bottom
fix             1 fixed_bottom setforce 0.0 0.0 0.0

# Thermostat (keep substrate at 600K)
fix             2 substrate nvt temp 600.0 600.0 0.1

# Deposition Region (above the surface)
region          depo_region block 0 25 0 25 15 20

# Deposit Fe atoms
fix             3 all deposit 10 3 100 12345 region depo_region near 1.0 vz -0.1 -0.5
# Deposit Pt atoms
fix             4 all deposit 10 4 100 23456 region depo_region near 1.0 vz -0.1 -0.5

# Time step
timestep        0.001

# Output
dump            1 all custom 100 dump.deposition.lammpstrj id type x y z
thermo          100

# Run
run             2000
"""

input_file = WORKDIR / "in.deposition"
with open(input_file, "w") as f:
    f.write(lammps_input)

print("LAMMPS Input written.")



## 4. Run Simulation

We execute LAMMPS. In CI mode, we skip the actual run if tools are missing, or run a dummy check.



In [ ]:
if IS_CI_MODE:
    print("CI Mode: Skipping actual LAMMPS execution.")
    # Create a dummy dump file for the next step
    (WORKDIR / "dump.deposition.lammpstrj").touch()
    # Create a dummy final structure
    slab.write(WORKDIR / "final_structure.xyz")

else:
    print("Running LAMMPS...")
    try:
        # Run LAMMPS
        # We redirect output to avoid cluttering the notebook
        with open(WORKDIR / "lammps.log", "w") as log:
            subprocess.run(["lmp", "-in", "in.deposition"], cwd=WORKDIR, stdout=log, stderr=log, check=True)
        print("LAMMPS execution complete.")
        
        # Parse output to get final structure
        # (For Tutorial 04, we need the final structure)
        # We can read the last frame of the dump
        try:
            from ase.io.lammpsrun import read_lammps_dump
            # Note: read_lammps_dump might be tricky depending on version
            # Alternatively use read(..., format='lammps-dump-text')
            traj = read(WORKDIR / "dump.deposition.lammpstrj", index=":", format="lammps-dump-text")
            final_atoms = traj[-1]
            final_atoms.write(WORKDIR / "final_structure.xyz")
            print("Final structure saved.")
        except Exception as e:
            print(f"Warning: Could not parse dump file: {e}")
            # Fallback: save the initial slab as final (better than nothing)
            slab.write(WORKDIR / "final_structure.xyz")
            
    except subprocess.CalledProcessError as e:
        print(f"LAMMPS failed. Check lammps.log. Error: {e}")
    except FileNotFoundError:
        print("LAMMPS executable 'lmp' not found.")
